In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from REINFORCE_helper import RunningVariance
from time import time
from REINFORCE_helper import BaseAgent
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, SGD
import keras.backend as K
import numpy as np

Using TensorFlow backend.


In [3]:
class ReinforceAgent(BaseAgent):
    # def __init__(self):
    def get_policy_model(self, lr=0.001, hidden_layer_neurons = 128, input_shape=[4], output_shape=2):
        ## Defino métrica - loss sin el retorno multiplicando
        def loss_metric(y_true, y_pred):
            y_true_norm = K.sign(y_true)
            return K.categorical_crossentropy(y_true_norm, y_pred)
        model = Sequential()
        model.add(Dense(hidden_layer_neurons, input_shape=input_shape, activation='relu'))
        model.add(Dense(output_shape, activation='softmax'))
        ## Por que la categorical_crossentropy funciona ok?
        model.compile(Adam(lr), loss=['categorical_crossentropy'], metrics=[loss_metric])
        return model
    
    def get_action(self, eval=False):
        p = self.model.predict([self.observation.reshape(1, self.nS)])
        if eval is False:
            action = np.random.choice(self.nA, p=p[0]) #np.nan_to_num(p[0])
        else:
            action = np.argmax(p[0])
        action_one_hot = np.zeros(self.nA)
        action_one_hot[action] = 1
        return action, action_one_hot, p
    
    def get_entropy(self, preds, epsilon=1e-12):
        entropy = np.mean(-np.sum(np.log(preds+epsilon)*preds, axis=1)/np.log(self.nA))
        return entropy
    
    def get_discounted_rewards(self, r):
        # Por si es una lista
        r = np.array(r, dtype=float)
        """Take 1D float array of rewards and compute discounted reward """
        discounted_r = np.zeros_like(r)
        running_add = 0
        for t in reversed(range(0, r.size)):
            running_add = running_add * self.gamma + r[t]
            discounted_r[t] = running_add
        return discounted_r 

In [4]:
reinforce_agent = ReinforceAgent('CartPole-v1', n_experience_episodes=1, EPISODES=2000, epochs=1, lr=0.001, algorithm='REINFORCE_CAUSAL')

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
obs, actions, preds, disc_sum_rews, rewards, ep_returns, ep_len, last_obs = reinforce_agent.get_experience_episodes()

In [6]:
disc_sum_rews

array([10.94516467,  9.95511979,  8.96408387,  7.97205593,  6.97903497,
        5.98501999,  4.99001   ,  3.994004  ,  2.997001  ,  1.999     ,
        1.        ])

In [7]:
# reinforce_agent = ReinforceAgent('Acrobot-v1', n_experience_episodes=1, EPISODES=2000, epochs=1, lr=0.001, algorithm='REINFORCE_CAUSAL')
# reinforce_agent = ReinforceAgent('LunarLander-v2', n_experience_episodes=50, EPISODES=2000, epochs=5, lr=0.0001, algorithm='REINFORCE_CAUSAL')
reinforce_agent = ReinforceAgent('MountainCar-v0', n_experience_episodes=50, EPISODES=2000, epochs=5, lr=0.0001, algorithm='REINFORCE_CAUSAL')

initial_time = time()
running_variance = RunningVariance()


while reinforce_agent.episode < reinforce_agent.EPISODES:
    obs, actions, preds, disc_sum_rews, rewards, ep_returns, ep_len, last_obs = reinforce_agent.get_experience_episodes()
    
    for dr in disc_sum_rews:
        running_variance.add(dr)

    pseudolabels = actions*disc_sum_rews.reshape(-1, 1)

    history = reinforce_agent.model.fit(obs, pseudolabels, verbose=0, epochs=reinforce_agent.epochs, batch_size=128)
    
    reinforce_agent.log_data(reinforce_agent.episode, 
                      history.history['loss'][0], 
                      np.mean(ep_len), 
                      reinforce_agent.get_entropy(preds), 
                      running_variance.get_variance(), 
                      history.history['loss_metric'][0], 
                      time() - initial_time, np.mean(ep_returns[-1]))

Instructions for updating:
Use tf.cast instead.
correr en linea de comando: tensorboard --logdir logs/
Episode: 51
Model on episode 52 improved from -inf to -181.3511705213644. Saved!
Episode: 103
Model on episode 104 improved from -181.3511705213644 to -181.3511705213644. Saved!
Episode: 155
Model on episode 156 improved from -181.3511705213644 to -181.3511705213644. Saved!
Episode: 207
Model on episode 208 improved from -181.3511705213644 to -181.3511705213644. Saved!
Episode: 259
Model on episode 260 improved from -181.3511705213644 to -181.3511705213644. Saved!
Episode: 311
Model on episode 312 improved from -181.3511705213644 to -181.3511705213644. Saved!
Episode: 363
Model on episode 364 improved from -181.3511705213644 to -181.3511705213644. Saved!
Episode: 415
Model on episode 416 improved from -181.3511705213644 to -181.3511705213644. Saved!
Episode: 467
Model on episode 468 improved from -181.3511705213644 to -181.3511705213644. Saved!
Episode: 519
Model on episode 520 improv

KeyboardInterrupt: 